In [1]:
library(randomForest)
library(sqldf)
library(ggplot2)
library(MASS)
library(pROC)
library(ROCR)

Warning message:
"package 'randomForest' was built under R version 3.3.3"randomForest 4.6-12
Type rfNews() to see new features/changes/bug fixes.
Warning message:
"package 'sqldf' was built under R version 3.3.3"Loading required package: gsubfn
Warning message:
"package 'gsubfn' was built under R version 3.3.3"Loading required package: proto
Could not load tcltk.  Will use slower R code instead.
Loading required package: RSQLite
Warning message:
"package 'RSQLite' was built under R version 3.3.3"
Attaching package: 'ggplot2'

The following object is masked from 'package:randomForest':

    margin

Warning message:
"package 'pROC' was built under R version 3.3.3"Type 'citation("pROC")' for a citation.

Attaching package: 'pROC'

The following objects are masked from 'package:stats':

    cov, smooth, var

Warning message:
"package 'ROCR' was built under R version 3.3.3"Loading required package: gplots
Warning message:
"package 'gplots' was built under R version 3.3.3"
Attaching package:

In [8]:
setwd("C:/Users/Gautam/Desktop/sumit1/results/historical/Q12005")   
set.seed(123)

In [9]:
abc="historical_data1_timeQ12005.csv" 
bcd="historical_data1_timeQ22005.csv"

In [10]:
 traindata <- read.csv.sql(file = abc, sql = "select * from file order by random() limit 500000")
#    testdata <- read.csv.sql(file = bcd, sql = "select * from file order by random() limit 50000")

Warning message:
"closing unused connection 4 (historical_data1_timeQ12005.csv)"

In [ ]:
#Converting columns into factors for training data
traindata$RepurchaseFlag <- as.factor(traindata$RepurchaseFlag)
traindata$ModificationFlag <- as.factor(traindata$ModificationFlag)
traindata$ZeroBalanceCode <- as.factor(traindata$ZeroBalanceCode)
traindata$CurrentLoadDelinquencyStatus <- as.factor(traindata$CurrentLoadDelinquencyStatus)

In [ ]:
#Converting columns into factors for testing data
testdata$RepurchaseFlag <- as.factor(testdata$RepurchaseFlag)
testdata$ModificationFlag <- as.factor(testdata$ModificationFlag)
testdata$ZeroBalanceCode <- as.factor(testdata$ZeroBalanceCode)
testdata$CurrentLoadDelinquencyStatus <- as.factor(testdata$CurrentLoadDelinquencyStatus)

In [ ]:
#Building modelfor the tree 
model_random<- randomForest(CurrentLoadDelinquencyStatus ~ CurrentActualUpb + LoanAge + RemainingMonthsToLegalMaturity +
                RepurchaseFlag + ModificationFlag + ZeroBalanceCode + CurrentInterestRate,
                data = traindata, ntree=70)

In [ ]:
importance(model_random)
varImpPlot(model_random)

In [ ]:
# Predicting the train data for the response.
PredictionWithClass<- predict(model_random,traindata,type='class')
t<- table(predictions = PredictionWithClass, actual = traindata$CurrentLoadDelinquencyStatus)
t

In [ ]:
#Calculating Accuracy through classification matrix
sum(diag(t))/sum(t)

In [ ]:
# Predicting the test data for the response.
PredictionWithClass<- predict(model_random,testdata,type='class')
t<- table(predictions = PredictionWithClass, actual = testdata$CurrentLoadDelinquencyStatus)
t

In [ ]:
#Calculating Accuracy through classification matrix
sum(diag(t))/sum(t)

In [ ]:
# ROC Curve for Train Data 
predictions=as.vector(model_random$votes[,1])
pred=prediction(predictions,traindata$CurrentLoadDelinquencyStatus)
perf_AUC=performance(pred,"auc") #Calculate the AUC value
AUC=perf_AUC@y.values[[1]]
perf_ROC=performance(pred,"tpr","fpr") #plot the actual ROC curve
plot(perf_ROC, main="ROC plot")
text(0.5,0.5,paste("AUC = ",format(AUC, digits=1, scientific=FALSE)))

In [ ]:
# ROC Curve for Train Data 
predictions=as.vector(model_random$votes[,1])
pred=prediction(predictions,testdata$CurrentLoadDelinquencyStatus)
perf_AUC=performance(pred,"auc") #Calculate the AUC value
AUC=perf_AUC@y.values[[1]]
perf_ROC=performance(pred,"tpr","fpr") #plot the actual ROC curve
plot(perf_ROC, main="ROC plot")
text(0.5,0.5,paste("AUC = ",format(AUC, digits=1, scientific=FALSE)))